In [2]:
import functools
from src.inference.vllm_inference import setup_model
from src.data.loader import load_benchmark
from src.evaluator import Evaluator
from src.metrics import PassAtk, GreedyPass, PercentPassed, MeanEntropy
from src.data.prompt import mbpp, humaneval
import datasets

In [3]:
llm, tokenizer, _ = setup_model(use_sft=True)

INFO 01-27 05:59:48 [utils.py:263] non-default args: {'trust_remote_code': True, 'seed': 42, 'max_model_len': 2048, 'enable_prefix_caching': False, 'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'enable_lora': True, 'max_lora_rank': 64, 'model': './models/qwen3-0.6b'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-27 05:59:57 [model.py:530] Resolved architecture: Qwen3ForCausalLM
INFO 01-27 05:59:57 [model.py:1545] Using max model len 2048
INFO 01-27 05:59:57 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-27 05:59:57 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-27 05:59:57 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
WARNING 01-27 05:59:58 [interface.py:470] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
(EngineCore_DP0 pid=69) INFO 01-27 05:59:58 [core.py:97] Initializing a V1 LLM engine (v0.14.1) with config: model='./models/qwen3-0.6b', speculative_config=None, tokenizer='./models/qwen3-0.6b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custo

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=69) INFO 01-27 06:00:21 [default_loader.py:291] Loading weights took 1.76 seconds
(EngineCore_DP0 pid=69) INFO 01-27 06:00:21 [punica_selector.py:20] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=69) INFO 01-27 06:00:22 [gpu_model_runner.py:3905] Model loading took 1.2 GiB memory and 20.811043 seconds
(EngineCore_DP0 pid=69) INFO 01-27 06:00:31 [backends.py:644] Using cache directory: /root/.cache/vllm/torch_compile_cache/f2a83223e6/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=69) INFO 01-27 06:00:31 [backends.py:704] Dynamo bytecode transform time: 9.47 s


...

(EngineCore_DP0 pid=69) INFO 01-27 06:00:40 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=69) INFO 01-27 06:00:47 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 12.03 s
(EngineCore_DP0 pid=69) INFO 01-27 06:00:47 [monitor.py:34] torch.compile takes 21.50 s in total
(EngineCore_DP0 pid=69) INFO 01-27 06:00:48 [gpu_worker.py:358] Available KV cache memory: 8.57 GiB
(EngineCore_DP0 pid=69) INFO 01-27 06:00:48 [kv_cache_utils.py:1305] GPU KV cache size: 80,208 tokens
(EngineCore_DP0 pid=69) INFO 01-27 06:00:48 [kv_cache_utils.py:1310] Maximum concurrency for 2,048 tokens per request: 39.16x


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE):   0%|          | 0/102 [00:00<?, ?it/s]

(EngineCore_DP0 pid=69) WARNING 01-27 06:00:49 [utils.py:268] Using default LoRA kernel configs


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 102/102 [00:27<00:00,  3.67it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 70/70 [00:04<00:00, 15.39it/s]


(EngineCore_DP0 pid=69) INFO 01-27 06:01:22 [gpu_model_runner.py:4856] Graph capturing finished in 33 secs, took 0.99 GiB
(EngineCore_DP0 pid=69) INFO 01-27 06:01:22 [core.py:273] init engine (profile, create kv cache, warmup model) took 60.04 seconds
INFO 01-27 06:01:22 [llm.py:347] Supported tasks: ['generate']


In [4]:
mbpp_mapper = functools.partial(mbpp.mbpp_to_task, tokenizer=tokenizer)
he_mapper = functools.partial(humaneval.humaneval_to_task, tokenizer=tokenizer)

In [5]:
raw_mbpp = datasets.load_dataset("google-research-datasets/mbpp", "sanitized", split="test")
raw_he = datasets.load_dataset("openai/openai_humaneval", split="test")

In [6]:
tasks_mbpp = load_benchmark(raw_mbpp, mbpp_mapper)
tasks_he = load_benchmark(raw_he, he_mapper)

In [7]:
metrics = [
	GreedyPass(),
    PassAtk(k=1, n_samples=1),
    PassAtk(k=10, n_samples=10),
	PercentPassed(),
    MeanEntropy(),
]

In [8]:
evaluator = Evaluator(llm, tokenizer, metrics)

In [9]:
print("=== MBPP RESULTS ===")
print(evaluator.run(tasks_mbpp))

=== MBPP RESULTS ===

🚀 Group: ['greedy@1']
⚙️ Params: n=1, temp=0.0, max_tokens=2048, logprobs=1


Adding requests:   0%|          | 0/257 [00:00<?, ?it/s]

WARNING 01-27 06:01:29 [input_processor.py:255] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Processed prompts:   0%|          | 0/257 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

🚀 Executing 257 samples in parallel using 8 workers...


...

Processed prompts:   0%|          | 0/2570 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

🚀 Executing 2570 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/2570 [00:00<?, ?it/s]

<string>:3: SyntaxWarning: invalid escape sequence '\S'
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:3: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


📊 pass@10 (n=10): 0.4280
{'greedy@1': 0.2918287937743191, 'pass@1 (n=1)': np.float64(0.30739299610894943), 'mean_%passed': np.float64(0.3550583657587549), 'mean_entropy': np.float64(0.0804614614155323), 'pass@10 (n=10)': np.float64(0.4280155642023346)}


In [ ]:
print("=== HUMANEVAL RESULTS ===")
print(evaluator.run(tasks_he))

=== HUMANEVAL RESULTS ===

🚀 Group: ['greedy@1']
⚙️ Params: n=1, temp=0.0, max_tokens=2048, logprobs=1


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

🚀 Executing 164 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/164 [00:00<?, ?it/s]

...

🚀 Executing 1640 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/1640 [00:00<?, ?it/s]

<string>:8: SyntaxWarning: invalid escape sequence '\/'
<string>:3: SyntaxWarning: invalid escape sequence '\w'


📊 pass@10 (n=10): 0.3598
{'greedy@1': 0.20121951219512196, 'pass@1 (n=1)': np.float64(0.20121951219512196), 'mean_%passed': np.float64(0.46469436220960614), 'mean_entropy': np.float64(0.06345893452322483), 'pass@10 (n=10)': np.float64(0.3597560975609756)}


ERROR 01-27 06:04:38 [core_client.py:610] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.
